In [1]:
import os
import numpy as np
import board
import pickle
import board as Board
import player
import game
from copy import deepcopy
import mcts
import matplotlib.pyplot as plt

from neural_network import*

In [2]:
player1 = player.Zero_Player('x', 'player1', nn_type="w", temperature=1)
player2 = player.Zero_Player('o', 'player2', nn_type="w", temperature=1)
players = [player1, player2]
tree = mcts.MCTS()
brd = Board.Board()

initial = np.array([[  0,   0,  0],
                 [  0,   0,   0],
                 [  0,   0,  0]])

In [11]:
import keras
from keras.layers import Input, Dense, Concatenate
from keras.models import Sequential, Model
from keras.layers import Dense, Conv1D, Flatten, Conv2D, MaxPooling1D, MaxPooling2D
from keras.optimizers import SGD
from keras import initializers
from keras.regularizers import l2

Input_1 = Input(shape=(3, 3, 1))

x1 = Conv2D(filters=4, kernel_size=(1, 3), activation='relu',
            kernel_regularizer=l2(0.0005),
            kernel_initializer=initializers.RandomNormal(stddev=0.1, mean=0),
            input_shape=(3, 3, 1))(Input_1)

x2 = Conv2D(filters=4, kernel_size=(3, 1), activation='relu',
            kernel_regularizer=l2(0.0005),
            kernel_initializer=initializers.RandomNormal(stddev=0.1, mean=0),
            input_shape=(3, 3, 1))(Input_1)

x3 = Conv2D(filters=16, kernel_size=(3, 3), activation='relu',
            kernel_regularizer=l2(0.0005),
            kernel_initializer=initializers.RandomNormal(stddev=0.1, mean=0),
            input_shape=(3, 3, 1))(Input_1)

x1 = MaxPooling2D((3, 1))(x1)
x2 = MaxPooling2D((1, 3))(x2)
x3 = MaxPooling2D((1, 1))(x3)

x = Concatenate()([x1, x2, x3])
x = Flatten()(x)

value_head = Dense(10, activation='relu')(x)
value_head = Dense(3, activation='softmax', name="V")(value_head)

policy_head = Dense(90, activation='relu')(x)
policy_head = Dense(10, activation='softmax', name="P")(policy_head)

model = Model(inputs=Input_1, outputs=[value_head, policy_head])
opt = SGD(lr=0.01, momentum=0.09)
model.compile(optimizer=opt,
              loss={"V": "categorical_crossentropy", "P":"categorical_crossentropy"},
              loss_weights = [1,0],
              metrics=['acc'])

In [12]:
def keras_copy(model):
    model_copy = keras.models.clone_model(model)
    model_copy.compile(optimizer=opt,
                       loss={"V": "categorical_crossentropy", "P": "categorical_crossentropy"},
                       loss_weights=[1, 0],
                       metrics=['acc'])
    model_copy.set_weights(model.get_weights())

    return model_copy

asdf = keras_copy(model)

In [4]:
brd.board = initial
PATH = {"path":[]}

self_play_game = game.Game(player1, player2, tree)
self_play_game.board = brd
self_play_results = self_play_game.play(100)
train_data = update_nn_training_set(tree.EDGES, tree.TREE)

In [13]:
X_clean = train_data[0]
Y_value = train_data[1]
# values are between 0 and 1, multiply by 2 to get range(0,2)
targets = (Y_value * 2).round()
Y_value_one_hot = np.eye(3)[targets.astype(int)]

Y_policy = train_data[2]

X_final = np.stack([X_clean], axis=-1)
model.fit(X_final, [Y_value_one_hot, Y_policy], epochs=100, verbose=1)

Epoch 1/100
172/172 [==============================] - 1s 1ms/step - loss: 1.0132 - V_loss: 1.0125 - P_loss: 1.9348 - V_acc: 0.4448 - P_acc: 0.0834
Epoch 2/100
172/172 [==============================] - 0s 1ms/step - loss: 0.9298 - V_loss: 0.9291 - P_loss: 1.9144 - V_acc: 0.5566 - P_acc: 0.0878
Epoch 3/100
172/172 [==============================] - 0s 1ms/step - loss: 0.8866 - V_loss: 0.8858 - P_loss: 1.9355 - V_acc: 0.5861 - P_acc: 0.0909
Epoch 4/100
172/172 [==============================] - 0s 1ms/step - loss: 0.8934 - V_loss: 0.8926 - P_loss: 1.9320 - V_acc: 0.5805 - P_acc: 0.0981
Epoch 5/100
172/172 [==============================] - 0s 1ms/step - loss: 0.8727 - V_loss: 0.8719 - P_loss: 1.9186 - V_acc: 0.5788 - P_acc: 0.0936
Epoch 6/100
172/172 [==============================] - 0s 1ms/step - loss: 0.8756 - V_loss: 0.8747 - P_loss: 1.9033 - V_acc: 0.5772 - P_acc: 0.0957
Epoch 7/100
172/172 [==============================] - 0s 1ms/step - loss: 0.8635 - V_loss: 0.8627 - P_loss: 1.9

KeyboardInterrupt: 

In [9]:
N_games = 100

player2.keras_nn = model
# player1.value_estimate ="nn"
player2.value_estimate="nn"

z_vs_r_game = game.Game(player1, player2, tree)
w1, w2 = z_vs_r_game.play_symmetric(N_games)
print('{} vs {} summary:'.format(player1.name, player2.name))
print('wins={}, draws={}, losses={}'.format(w1, N_games-w1-w2, w2))

KeyboardInterrupt: 

In [7]:
model_copy= keras.models.clone_model(model)
model_copy.build((None, 10)) # replace 10 with number of variables in input layer
model_copy.compile(optimizer=opt,
              loss={"V": "categorical_crossentropy", "P":"categorical_crossentropy"},
              loss_weights = [1,0],
              metrics=['acc'])
model_copy.set_weights(model.get_weights())